# PyTorch中如何进行向量微分、矩阵微分与计算雅可比行列式

来自b站up主deep_thoughts 合集【PyTorch源码教程与前沿人工智能算法复现讲解】

P_11_PyTorch中如何进行向量微分、矩阵微分与计算雅可比行列式：
    
https://www.bilibili.com/video/BV1Xr4y1r7wa/?spm_id_from=pageDriver&vd_source=18e91d849da09d846f771c89a366ed40

autograd 官方文档：https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html

jacobian 官方文档：https://pytorch.org/docs/stable/generated/torch.autograd.functional.jacobian.html

## 向量对向量

In [4]:
import torch
from torch.autograd.functional import jacobian

def func(x):
    return x.exp().sum(dim=1)

x = torch.randn(2,3)
print(x)
y = func(x)
print(y)
jacobian(func, x)

tensor([[-1.1897,  0.1149, -0.7102],
        [ 2.3557, -0.0798, -1.4557]])
tensor([ 1.9176, 11.7021])


tensor([[[ 0.3043,  1.1217,  0.4915],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [10.5456,  0.9233,  0.2332]]])

In [6]:
a = torch.randn(3)
print(a)

def func(x):
    return a+x

x = torch.randn(3)
print(x)

jacobian(func, x)

tensor([-0.9941,  0.0499, -0.8455])
tensor([ 0.1459,  0.0730, -1.1449])


tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [7]:
x = torch.randn(3, requires_grad=True)

y = func(x)
print(y)
y.backward(torch.ones_like(y))
print(x.grad)

tensor([-1.1797, -1.2804, -2.7275], grad_fn=<AddBackward0>)
tensor([1., 1., 1.])


In [10]:
torch.ones_like(y) @ jacobian(func, x)  # @矩阵相乘

tensor([1., 1., 1.])

## 矩阵对矩阵
退化成向量梯度算

In [11]:
a = torch.randn(2, 3, requires_grad=True)
b = torch.randn(3, 2, requires_grad=True)
print(a)
print(b)

tensor([[ 0.7477, -1.0058, -2.2361],
        [-0.2903, -0.3600, -0.5330]], requires_grad=True)
tensor([[-2.1878,  0.5005],
        [ 0.2652, -1.4495],
        [-0.5458,  0.8088]], requires_grad=True)


In [13]:
y = a @ b
print(y)

tensor([[-0.6822,  0.0234],
        [ 0.8305, -0.0546]], grad_fn=<MmBackward0>)


In [15]:
y.backward(torch.ones_like(y))

In [16]:
print(a.grad)
print(b.grad)

tensor([[-1.6873, -1.1843,  0.2631],
        [-1.6873, -1.1843,  0.2631]])
tensor([[ 0.4575,  0.4575],
        [-1.3658, -1.3658],
        [-2.7692, -2.7692]])


In [19]:
def func(a):
    return a @ b

torch.ones_like(func(a[0])) @ jacobian(func, a[0])  # 求a的第一行梯度

tensor([-1.6873, -1.1843,  0.2631])

In [18]:
torch.ones_like(func(a[1])) @ jacobian(func, a[1])  # 求a的第二行梯度

tensor([-1.6873, -1.1843,  0.2631])

In [20]:
def func(b):
    return a @ b

torch.ones_like(func(b[:, 0])) @ jacobian(func, b[:, 0])  # 求b的第一列梯度

tensor([ 0.4575, -1.3658, -2.7692])

In [21]:
torch.ones_like(func(b[:, 1])) @ jacobian(func, b[:, 1])  # 求b的第二列梯度

tensor([ 0.4575, -1.3658, -2.7692])